In [2]:
from polygon import RESTClient
from local_settings import polygon as settings

from datetime import date,datetime
from typingOptional import Any,Optional
import pandas as pd
from requests.adapters import GTTPAdapter
from urllib3.util.retry import Retry

markets = ['stocks','crypto','fx']
class MyRESTClient(RESTClient):
    def __init__(self,auth_key=settings['api_key'],timeout= 5):
        super.__init__(auth_key)
        retry_strategy = Retry(total = 10, backoff_factor = 10, status_forcelist = [429,500,501,503,504])
        adapter = HTTPAdapter(max_retries = retry_strategy)
        self._session.mount('https://',adapter)

    def get_tickers(self, market:str=None) -> pd.DataFrame:
        if not market in markets:
            raise Exception(f'Market must be one of {markets}.')

        resp = self.reference_tickers_v3(market=market)
        if hasattr(resp, 'results'):
            df = pd.DataFrame(resp.results)

            while hasattr(resp, 'next_url'):
                resp = self.reference_tickers_v3(next_url=resp.next_url)
                df = df.append(pd.DataFrame(resp.results))

            if market == 'crypto':
                df = df[df['currency_symbol'] == 'USD']
                df['name'] = df['base_currency_name']
                df = df[['ticker', 'name', 'market', 'active']]

            df = df.drop_duplicates(subset='ticker')
            return df
        return None

client = MyRESTClient(settings['api_key'])
df = client.get_tickers(market='crypto')



ModuleNotFoundError: No module named 'local_settings'

In [ ]:
df
